In [104]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import skew
import numpy as np
import seaborn as sns
import glob
import re

In [111]:
#!rm -r Results
!g++ Proyect.cpp -o ICDF.out
#!mkdir Results

In [112]:
#Corremos la simulacion
!./ICDF.out

In [113]:
#Lectura Alternativa del DF
DF = pd.read_csv('./Results/_South_Complete_trend_analysis.dat', sep = ' ')

In [114]:
#Sacamos el N mas frecuente
N_most_frequent = DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]
N = N_most_frequent["#Autos"].mode()[0]
iterations = N_most_frequent[N_most_frequent['#Autos'] == N].Iteration.unique()

In [115]:
print(DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']].mean())
print(DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']].std())

Iteration    24999.50000
#Autos         713.31544
dtype: float64
Iteration    14433.901067
#Autos          38.132459
dtype: float64


In [116]:
DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]['#Autos'].value_counts()

727    1411
754    1165
718    1158
703    1155
758    1086
       ... 
755     332
694     330
780     324
791     318
698     280
Name: #Autos, Length: 88, dtype: int64

In [117]:
#Seleccionamos todos los procesos que contengan al N mas frecuente
DF_ = DF[DF.Iteration.isin(iterations)]

In [65]:
#Revisamos las estadisticas de los resultados 
stats = pd.pivot_table(DF_, index = 'Iteration', values = 'Delta_t', aggfunc = [np.mean,np.std, np.median, skew])

In [66]:
stats.sort_values(('std','Delta_t'))

,mean,std,median,skew
,Delta_t,Delta_t,Delta_t,Delta_t
Iteration,,,,
16987,4.828496,4.983881,3.32906,3.007494
16735,4.828496,4.983881,3.32906,3.007494
16736,4.828496,4.983881,3.32906,3.007494
16737,4.828496,4.983881,3.32906,3.007494
16738,4.828496,4.983881,3.32906,3.007494
...,...,...,...,...
13259,4.831442,6.444998,3.31496,7.231194
13260,4.831442,6.444998,3.31496,7.231194


In [ ]:
#Seleccionamos una simulacion que tenga resultados mas cercanos a los reales

In [ ]:
DF_ = DF_[DF_.Iteration.isin([25930])]

In [ ]:
#Hacemos el trend analysis de la region North
#Ajustamos Minimos cuadrados
x = np.array(DF_['#Autos']).reshape(-1, 1)
y = np.array(DF_['Delta_t']).reshape(-1, 1)
linreg = LinearRegression().fit(x,y)

In [ ]:
print('Intercept: {}'.format(linreg.intercept_))
print('Coef: {}'.format(linreg.coef_))

In [ ]:
plt.scatter(x,y)
ax = plt.gca()
ax.set_xlabel('Vehicle Count')
ax.set_ylabel('Time Intervals')
plt.plot(x,linreg.intercept_+linreg.coef_*x, color='r')

# NO BORRAR

#Creamos lista de DF
DF = pd.DataFrame()
for i in glob.glob('./Results/*.dat'):
    data = pd.read_csv(i, sep = " ")
    data['Iteration'] = int(re.findall(r'\d+',i)[0])
    DF =  pd.concat([DF,data],ignore_index=True)